# Imports

In [1]:
import vertexai
import os
import torch
import json
from vertexai.generative_models import Tool
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.globals import set_debug, set_verbose
from langchain.memory import ConversationBufferMemory
import requests
from typing import Dict
from langchain_google_vertexai import HarmBlockThreshold, HarmCategory
from pyowm import OWM
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
from langchain.tools import tool
from habanero import Crossref
from langchain.agents import initialize_agent, AgentType
from langchain_google_vertexai import VertexAI

In [2]:
from Agents.WeatherAgent import WeatherAgent
from Agents.CrossrefAgent import CrossrefAgent
from Agents.ElsevierAgent import ElsevierAgent
from Agents.DBAgent import DBAgent
from CustomAgentRouter import CustomAgentRouter
from HITLAgentRouter import HITLAgentRouter

/home/amartinez/MARTINEZ_Alex_Llm-Agents_2024/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Cache

In [3]:
# Optimize the memory usage

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
torch.cuda.empty_cache()

# API Keys

In [4]:
# Set the API keys for various tools

os.environ["OPENWEATHERMAP_API_KEY"] = "54b1d6c1af466db06bee6219ddb33f7c"
INFLUXDB_URL = "https://apiivm78.etsii.upm.es:8086"
INFLUXDB_TOKEN = "bYNCMsvuiCEoJfPFL5gPgWgDISh79wO4dH9dF_y6cvOKp6uWTRZHtPIwEbRVb2gfFqo3AdygZCQIdbAGBfd31Q=="
INFLUXDB_ORG = "UPM"
INFLUXDB_BUCKET = "LoraWAN"

# Model Set up

In [5]:
vertexai.init(project="summer-surface-443821-r9", location="europe-southwest1")

model = "gemini-1.5-flash"

In [6]:
safety_settings ={
    HarmCategory.HARM_CATEGORY_UNSPECIFIED: HarmBlockThreshold.BLOCK_NONE,
    HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_HATE_SPEECH: HarmBlockThreshold.BLOCK_ONLY_HIGH,
    HarmCategory.HARM_CATEGORY_HARASSMENT: HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: HarmBlockThreshold.BLOCK_NONE,
    }

In [7]:
model_kwargs = {
    # temperature (float): The sampling temperature controls the degree of
    # randomness in token selection.
    "temperature": 0.28,
    # max_output_tokens (int): The token limit determines the maximum amount of
    # text output from one prompt.
    "max_output_tokens": 1000,
    # top_p (float): Tokens are selected from most probable to least until
    # the sum of their probabilities equals the top-p value.
    "top_p": 0.95,
    # top_k (int): The next token is selected from among the top-k most
    # probable tokens.
    "top_k": 40,
    # safety_settings (Dict[HarmCategory, HarmBlockThreshold]): The safety
    # settings to use for generating content.
    "safety_settings": safety_settings,
}

## Memory

In [8]:
# Define memory buffer

memory = ConversationBufferMemory(
        memory_key="chat_history",
        return_messages=True,
    )

/tmp/ipykernel_3930683/2280263513.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


# Templates

In [ ]:
# Define the answer structure

PREFIX = """Answer the following questions as best you can. You have access to the following tools:"""
FORMAT_INSTRUCTIONS = """To use a tool, please use the following format:

```
Thought: Do I need to use a tool? Yes
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
```

When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

```
Thought: Do I need to use a tool? No
Final Answer: [your response here]
```"""
SUFFIX = """Begin!
Question: {input}
Thought:{agent_scratchpad}"""

In [ ]:
# Define a prompt template

template = "\n\n".join([PREFIX, "{tools}", FORMAT_INSTRUCTIONS, SUFFIX])
prompt = PromptTemplate(template=template, input_variables=["input", "chat_history", "tools", "tool_names", "agent_scratchpad"])
print(prompt)

# Tools

### OpenWeatherMap

In [ ]:
# OpenWeatherMap configuration
API_KEY = "54b1d6c1af466db06bee6219ddb33f7c"
owm = OWM(API_KEY)
mgr = owm.weather_manager()

# Define a function to obtain climate
def get_weather(location: str) -> str:
    try:
        observation = mgr.weather_at_place(location)
        weather = observation.weather
        return (
            f"The current weather in {location} is {weather.status} with a temperature of "
            f"{weather.temperature('celsius')['temp']}°C."
        )
    except Exception as e:
        return f"Failed to fetch weather data for {location}: {str(e)}"

# Create compatible tool
weather_tool = Tool(
    name="get_weather",
    func=get_weather,
    description="Fetch the current weather for a specified location."
)

### InfluxDB

In [31]:
from influxdb_client import InfluxDBClient
from langchain.tools import Tool
import re
import json
from pydantic import BaseModel
from langchain.tools.base import StructuredTool

# Client configuration
INFLUXDB_URL = "https://apiivm78.etsii.upm.es:8086"
INFLUXDB_TOKEN = "bYNCMsvuiCEoJfPFL5gPgWgDISh79wO4dH9dF_y6cvOKp6uWTRZHtPIwEbRVb2gfFqo3AdygZCQIdbAGBfd31Q=="
INFLUXDB_ORG = "UPM"
INFLUXDB_BUCKET = "LoraWAN"

client = InfluxDBClient(url=INFLUXDB_URL, token=INFLUXDB_TOKEN, org=INFLUXDB_ORG)
query_api = client.query_api()

# Supported parameters
VALID_METRICS = {"temperature", "humidity", "light", "motion", "vdd"}
VALID_AGGREGATIONS = {"mean", "max", "min", "sum"}

# Define expected input schema for the tool using Pydantic
class InfluxDBQueryInput(BaseModel):
    """
    Defines the input parameters required for querying InfluxDB.
    """
    metric: str          # Sensor metric to query (e.g., temperature, humidity)
    time_range: str      # Time range for the query (e.g., 24h, 7d)
    aggregation: str     # Aggregation function (e.g., mean, max)

# Function to construct Flux query dynamically
def construct_flux_query(params: dict) -> str:
    """
    Constructs a Flux query based on extracted parameters.
    
    Args:
        params (dict): A dictionary containing 'metric', 'time_range', and 'aggregation'.
    
    Returns:
        str: A formatted Flux query.
    """
    measurement = "sensor_data"  # Default measurement
    field = params.get("metric", "humidity")  # Default metric
    time_range = params.get("time_range", "24h")  # Default to 24h if missing
    aggregation = params.get("aggregation", "mean")  # Default to mean

    # Validate metric
    if field not in VALID_METRICS:
        raise ValueError(f"❌ Invalid metric '{field}'. Available metrics: {', '.join(VALID_METRICS)}")

    # Validate aggregation function
    if aggregation not in VALID_AGGREGATIONS:
        raise ValueError(f"❌ Invalid aggregation '{aggregation}'. Available functions: {', '.join(VALID_AGGREGATIONS)}")

    # Construct Flux query
    flux_query = f"""
    from(bucket: "{INFLUXDB_BUCKET}")
      |> range(start: -{time_range})
      |> filter(fn: (r) => r["_measurement"] == "{measurement}")
      |> filter(fn: (r) => r["_field"] == "{field}")
      |> aggregateWindow(every: 1h, fn: {aggregation}, createEmpty: false)
      |> yield(name: "result")
    """
    return flux_query

# Function that receives individual arguments (required by StructuredTool)
def query_influxdb(metric: str, time_range: str, aggregation: str) -> str:
    """
    StructuredTool-compatible function to query InfluxDB using individual parameters.

    Args:
        metric (str): Sensor metric to query (e.g., "temperature", "humidity").
        time_range (str): Time range for the query (e.g., "24h", "7d").
        aggregation (str): Aggregation function to apply (e.g., "mean", "max").

    Returns:
        str: Formatted result or error message.
    """
    params = {
        "metric": metric,
        "time_range": time_range,
        "aggregation": aggregation
    }
    return _query_influxdb_internal(params)

# Internal function to perform the actual query logic
def _query_influxdb_internal(params: dict) -> str:
    """
    Constructs and executes a Flux query on InfluxDB from parameter dictionary.

    Args:
        params (dict): Dictionary containing 'metric', 'time_range', and 'aggregation'.

    Returns:
        str: Query results or an error message.
    """
    try:
        # Build the Flux query dynamically from parameters
        flux_query = construct_flux_query(params)

        print(f"📊 Extracted Parameters: {params}")
        print(f"🔥 Executing Flux Query:\n{flux_query}") 

        # Execute the query using InfluxDB client
        result = query_api.query(org=INFLUXDB_ORG, query=flux_query)
        results = []

        # Format the results
        for table in result:
            for record in table.records:
                results.append(f"Time: {record.get_time()}, Value: {record.get_value()}")

        return "\n".join(results) if results else "⚠️ No data found in the database. Verify if data exists for this time range."
    
    except ValueError as ve:
        return str(ve)  # Return validation error messages

    except Exception as e:
        return f"❌ Error querying InfluxDB: {str(e)}"

# Function to extract the time range from a query
def extract_time_range(user_query: str) -> str:
    """
    Extracts the time range from a user query.

    Args:
        user_query (str): The input query from the user.

    Returns:
        str: A formatted time range for InfluxDB (e.g., "24h", "7d", "30d").
    """
    time_patterns = {
        r"(\d+)\s*(minute|minutes|min)": "m",
        r"(\d+)\s*(hour|hours|h)": "h",
        r"(\d+)\s*(day|days|d)": "d",
        r"(\d+)\s*(week|weeks|w)": "w",
        r"(\d+)\s*(month|months|mo)": "d",  # Approximate: 1 month = 30 days
        r"(\d+)\s*(year|years|y)": "d"  # Approximate: 1 year = 365 days
    }

    detected_range = "24h"  # Default if no time range is found

    for pattern, unit in time_patterns.items():
        match = re.search(pattern, user_query, re.IGNORECASE)
        if match:
            value = int(match.group(1))  # Extract numeric value
            if "month" in pattern:
                value *= 30  # Convert months to days
            elif "year" in pattern:
                value *= 365  # Convert years to days
            detected_range = f"{value}{unit}"
            break

    return detected_range

# LangChain compatible tool

influx_tool = StructuredTool.from_function(
    name="InfluxDB Query Tool",
    description=(
        "Fetches sensor data from InfluxDB. "
        "Requires parameters like metric (e.g., humidity, temperature), "
        "time_range (e.g., 24h), and aggregation (e.g., mean)."
    ),
    func=query_influxdb,
    args_schema=InfluxDBQueryInput
)

In [32]:
result = query_influxdb(metric="temperature", time_range="18d", aggregation="mean")
print(result)

📊 Extracted Parameters: {'metric': 'temperature', 'time_range': '18d', 'aggregation': 'mean'}
🔥 Executing Flux Query:

    from(bucket: "LoraWAN")
      |> range(start: -18d)
      |> filter(fn: (r) => r["_measurement"] == "sensor_data")
      |> filter(fn: (r) => r["_field"] == "temperature")
      |> aggregateWindow(every: 1h, fn: mean, createEmpty: false)
      |> yield(name: "result")
    
Time: 2025-03-22 17:00:00+00:00, Value: 14.9
Time: 2025-03-22 18:00:00+00:00, Value: 14.91666666666667
Time: 2025-03-22 19:00:00+00:00, Value: 14.912500000000001
Time: 2025-03-22 20:00:00+00:00, Value: 14.863636363636367
Time: 2025-03-22 21:00:00+00:00, Value: 14.8
Time: 2025-03-22 22:00:00+00:00, Value: 14.774999999999997
Time: 2025-03-22 23:00:00+00:00, Value: 14.733333333333333
Time: 2025-03-23 00:00:00+00:00, Value: 14.688888888888888
Time: 2025-03-23 01:00:00+00:00, Value: 14.633333333333331
Time: 2025-03-23 02:00:00+00:00, Value: 14.618181818181817
Time: 2025-03-23 03:00:00+00:00, Value: 14

### Elsevier

In [ ]:
def get_article_content(title: str) -> str:
    """
    Fetch the content of a specific article using Elsevier's APIs based on its title.

    Args:
    - title (str): The title of the article to search for.

    Returns:
    - str: The content or abstract of the specified article.
    """

    API_KEY = "87ab69edd16f0cdb92e611b99b8f4ee6"
    BASE_SEARCH_URL = "https://api.elsevier.com/content/search/scopus"
    BASE_ARTICLE_URL = "https://api.elsevier.com/content/article/doi"
    BASE_ARTICLE_EID_URL = "https://api.elsevier.com/content/article/eid"

    # 1. Search article by title
    headers = {
        "Accept": "application/json",
        "X-ELS-APIKey": API_KEY,
    }
    params = {
        "query": f"TITLE({title})",
        "count": 1,
    }
    
    search_response = requests.get(BASE_SEARCH_URL, headers=headers, params=params)

    if search_response.status_code != 200:
        return f"Error: No se pudo buscar el artículo. Código de estado: {search_response.status_code}, Error: {search_response.text}"

    search_data = search_response.json()
    entries = search_data.get("search-results", {}).get("entry", [])

    if not entries:
        return f"No se encontró ningún artículo con el título '{title}'."

    # 2. Fetch article identifier
    article_entry = entries[0]
    print("Artículo encontrado:", article_entry)  # Debugging info

    doi = article_entry.get("prism:doi")
    eid = article_entry.get("eid")

    if not doi and not eid:
        return f"No se encontró DOI ni EID para el artículo con el título '{title}'. No se puede recuperar el contenido."

    # 3. Fetch article content using DOI
    if doi:
        article_url = f"{BASE_ARTICLE_URL}/{doi}"
    else:
        article_url = f"{BASE_ARTICLE_EID_URL}/{eid}"

    print(f"Intentando recuperar el artículo desde: {article_url}")  # Debugging info

    article_response = requests.get(article_url, headers=headers)

    if article_response.status_code != 200:
        return f"Error: No se pudo recuperar el contenido del artículo. Código de estado: {article_response.status_code}, Error: {article_response.text}"

    article_data = article_response.json()

    abstract = article_data.get("full-text-retrieval-response", {}).get("coredata", {}).get("dc:description", "No abstract found")

    return f"Contenido del artículo '{title}':\n\n{abstract}"


In [ ]:
title = "Taking the next step with generative artificial intelligence: The transformative role of multimodal large language models in science education"
content = get_article_content(title)
print(content)

### Crossref

In [ ]:
def crossref(subject: str) -> Dict:
    limit = 5
    results = {}
    cr = Crossref()
    try:
        result = cr.works(query=subject, limit=limit)
        for i in range(0, limit-1):
            title = result['message']['items'][i].get('title', ['No Title'])[0]
            abstract = result['message']['items'][i].get('abstract', 'No abstract available')
            results[title] = abstract
        return results
    except Exception as e:
        return {"error": str(e)}

crossref_tool = Tool(
    name="Crossref article search",
    func=crossref,
    description="Use this to fetch article titles and abstracts based on a given subject.",
)


In [ ]:
cr = Crossref()
x = cr.works(query = "ecology")
x['message']['items'][4]['abstract']

In [ ]:
a = crossref(subject="computing")

print(a)

# Agents

## Weather Agent

In [ ]:
weather_agent = WeatherAgent()

print("Registered tools:", weather_agent.agent.tools)

In [ ]:
weather_agent.run("Is it raining in Manchester?")

## DB Agent 

In [9]:
DB_agent = DBAgent()

print("Registered tools:", DB_agent.agent.tools)

Registered tools: [StructuredTool(name='InfluxDB Query Tool', description='Fetches sensor data from InfluxDB. Requires parameters like metric (e.g., humidity, temperature), time_range (e.g., 24h), and aggregation (e.g., mean).', args_schema=<class 'Tools.DBTool.InfluxDBQueryInput'>, func=<function query_influxdb at 0x7bd5f81a9940>)]


/mnt/sdb1/home/amartinez/MARTINEZ_Alex_Llm-Agents_2024/src/tfg/Agents/BaseAgent.py:44: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  self.agent = initialize_agent(


In [12]:
DB_agent.run("What was the max humidity of the last 20 days") 

🚀 Query being processed by agent:
What was the max humidity of the last 20 days


> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "InfluxDB Query Tool",
  "action_input": {
    "metric": "humidity",
    "time_range": "20d",
    "aggregation": "max"
  }
}
```📊 Extracted Parameters: {'metric': 'humidity', 'time_range': '20d', 'aggregation': 'max'}
🔥 Executing Flux Query:

    from(bucket: "LoraWAN")
      |> range(start: -20d)
      |> filter(fn: (r) => r["_measurement"] == "sensor_data")
      |> filter(fn: (r) => r["_field"] == "humidity")
      |> aggregateWindow(every: 1h, fn: max, createEmpty: false)
      |> yield(name: "result")
    

Observation: Time: 2025-03-20 18:00:00+00:00, Value: 52.0
Time: 2025-03-20 19:00:00+00:00, Value: 55.0
Time: 2025-03-20 20:00:00+00:00, Value: 58.0
Time: 2025-03-20 21:00:00+00:00, Value: 60.0
Time: 2025-03-20 22:00:00+00:00, Value: 61.0
Time: 2025-03-20 23:00:00+00:00, Value: 61.0
Time: 2025-03-21 00:00:00+00:00, Value: 61.0
Time

'The maximum humidity over the last 20 days was 63.0%'

## Elsevier Agent

In [13]:
elsevier_agent = ElsevierAgent()

In [14]:
elsevier_agent.run("Search some articles about AI")

🚀 Query being processed by agent:
Search some articles about AI


> Entering new AgentExecutor chain...
Action:
```json
{
  "action": "Elsevier Article Search",
  "action_input": "Artificial Intelligence"
}
```
Observation: Error: Unable to retrieve the article content. Status code: 404, Error: {"service-error":{"status":{"statusCode":"RESOURCE_NOT_FOUND","statusText":"The resource specified cannot be found."}}}
Thought:Action:
```json
{
  "action": "Elsevier Article Search",
  "action_input": "Artificial Intelligence in Healthcare"
}
```

Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-flash. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..



Observation: Error: Unable to retrieve the article content. Status code: 404, Error: {"service-error":{"status":{"statusCode":"RESOURCE_NOT_FOUND","statusText":"The resource specified cannot be found."}}}
Thought:

Retrying langchain_google_vertexai.llms._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised ResourceExhausted: 429 Quota exceeded for aiplatform.googleapis.com/generate_content_requests_per_minute_per_project_per_base_model with base model: gemini-1.5-flash. Please submit a quota increase request. https://cloud.google.com/vertex-ai/docs/generative-ai/quotas-genai..


KeyboardInterrupt: 

## Crossref Agent

In [ ]:
crossref_agent = CrossrefAgent()

In [ ]:
crossref_agent.run("I want to know more about quantum computing, can you help me?")

In [ ]:
# Agents dictionary
agents = {
    "weather": weather_agent,
    "article title": crossref_agent,
    "article content": elsevier_agent,
    "generic": weather_agent,  
}

# Agent Router

In [ ]:
# Initialize the router
router = CustomAgentRouter()

In [ ]:
inputs = [
    "What is the weather like in Madrid?",  # Should go to WeatherAgent
    "Search for articles about machine learning.",  # Should go to CrossrefAgent
    "I need the content of an article titled 'Deep Learning Basics'."  # Should go to ElsevierAgent
]

In [ ]:
from transformers import pipeline
pipeline = pipeline("text-classification", model="distilbert-base-uncased", top_k=None)

In [ ]:
predictions = pipeline("What is the weather like in Madrid?")
print("Predictions output:", predictions)

In [ ]:
response = router.run( "What is the weather like in Madrid?")
print(f"Response: {response}\n")

In [ ]:
for input_text in inputs:
    print(f"Input: {input_text}")
    response = router.run(input_text)
    print(f"Response: {response}\n")

# Tests

In [3]:
from langgraph.graph import StateGraph, START, END
from typing import Dict, Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langchain.memory import ConversationBufferMemory
from CustomAgentRouter import CustomAgentRouter
from Agents.WeatherAgent import WeatherAgent
from Agents.DBAgent import DBAgent
from Agents.CrossrefAgent import CrossrefAgent
from Agents.ElsevierAgent import ElsevierAgent
import transformers
from langchain.schema import HumanMessage
transformers.logging.set_verbosity_error()
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Define the state structure with memory
class State(TypedDict):
    messages: Annotated[list, add_messages]
    route: str
    memory: ConversationBufferMemory

def classify_query(state: State) -> Dict[str, str]:
    """
    Uses CustomAgentRouter to classify user input and determine the correct agent.
    """
    user_input = state["messages"][-1].content
    router = CustomAgentRouter()
    category, _ = router.classify_text(user_input)
    return {"route": category}

def route_decision(state: State) -> str:
    """
    Determines the next node based on classification result.
    """
    valid_routes = {"weather", "database", "article title", "article content"}
    return state["route"] if state["route"] in valid_routes else "generic"

# Agent functions with memory integration
def weather_agent(state: State) -> Dict[str, str]:
    user_input = state["messages"][-1].content
    agent = WeatherAgent()
    response = agent.run(user_input)
    state["memory"].save_context({"user_input": user_input}, {"response": response})
    return {"messages": state["messages"] + [{"role": "assistant", "content": response}]}

def db_agent(state: State) -> Dict[str, str]:
    user_input = state["messages"][-1].content
    agent = DBAgent()
    response = agent.run(user_input)
    state["memory"].save_context({"user_input": user_input}, {"response": response})
    return {"messages": state["messages"] + [{"role": "assistant", "content": response}]}

def crossref_agent(state: State) -> Dict[str, str]:
    user_input = state["messages"][-1].content
    agent = CrossrefAgent()
    response = agent.run(user_input)
    state["memory"].save_context({"user_input": user_input}, {"response": response})
    return {"messages": state["messages"] + [{"role": "assistant", "content": response}]}

def elsevier_agent(state: State) -> Dict[str, str]:
    user_input = state["messages"][-1].content
    agent = ElsevierAgent()
    response = agent.run(user_input)
    state["memory"].save_context({"user_input": user_input}, {"response": response})
    return {"messages": state["messages"] + [{"role": "assistant", "content": response}]}

# Define the LangGraph workflow
graph = StateGraph(State)

graph.add_node("classify", classify_query)
graph.add_node("weather", weather_agent)
graph.add_node("database", db_agent)
graph.add_node("crossref", crossref_agent)
graph.add_node("elsevier", elsevier_agent)
graph.add_node("generic", lambda state: {"messages": state["messages"] + [{"role": "assistant", "content": "I'm not sure how to help with that."}]})

graph.add_edge(START, "classify")
graph.add_conditional_edges("classify", route_decision, {
    "weather": "weather",
    "database": "database",
    "article title": "crossref",
    "article content": "elsevier",
    "generic": "generic"
})

graph.add_edge("weather", END)
graph.add_edge("database", END)
graph.add_edge("crossref", END)
graph.add_edge("elsevier", END)
graph.add_edge("generic", END)

compiled_graph = graph.compile()

def run_conversation(user_input: str):
    state = {
        "messages": [{"role": "user", "content": user_input}],
        "memory": ConversationBufferMemory()
    }
    for event in compiled_graph.stream(state):
        for value in event.values():
            if "messages" in value:
                print("Assistant:", value["messages"][-1]["content"])
            else:
                print("Assistant: (No response received)")

In [ ]:
while True:
        user_input = input("User: ")
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)